The notebook updates error arrays for 1D and 2D extracted spectra (x1d and s2d products) from Poisson and read noise variance extensions. Use this workaround until reference flat files are delivered with associated error estimates.

In [ ]:
import requests

import numpy as np
from matplotlib import pyplot as plt
from astropy.io import fits

import warnings
warnings.simplefilter('ignore', RuntimeWarning)

Define a function to download a named file via the MAST API. The function includes authentication logic, but this example uses public data, so no MAST API token is required.

In [ ]:
def get_jwst_file(name, mast_api_token=None):
    """Retrieve a JWST data file from MAST archive."""
    mast_url = "https://mast.stsci.edu/api/v0.1/Download/file"
    params = dict(uri=f"mast:JWST/product/{name}")
    if mast_api_token:
        headers = dict(Authorization=f"token {mast_api_token}")
    else:
        headers = {}
    r = requests.get(mast_url, params=params, headers=headers, stream=True)
    r.raise_for_status()
    with open(name, "wb") as fobj:
        for chunk in r.iter_content(chunk_size=1024000):
            fobj.write(chunk)

Create a list of filenames. Download the data from MAST.

In [ ]:
x1d_names = []
s2d_names = []
for activity in [2]:
    for exposure in [1]:
        for detector in [1, 2]:
            x1d_names.append(f'jw02288003001_041{activity:02}_{exposure:05}_nrs{detector}_x1d.fits')
            s2d_names.append(f'jw02288003001_041{activity:02}_{exposure:05}_nrs{detector}_s2d.fits')

for name in x1d_names + s2d_names:
    print(f'Downloading {name}')
    get_jwst_file(name)

For the extracted spectrum, error information is stored in table columns.  The overall error on the flux is reported in the FLUX_ERROR column.  For some versions of the pipeline and NIRSpec flat reference files, the FLUX_ERROR may be erroneously filled with NaN values.

The error on the flux may be recalculated from the component variance sources.  These are stored in the spectral columns FLUX_VAR_POISSON, FLUX_VAR_RNOISE, and FLUX_VAR_FLAT.  The FLUX_VAR_FLAT may be filled with zeroes or NaN values, for flat reference files that do not yet have associated error estimates.

Combine the three variance sources in quadrature and take the square root to estimate the error on the flux.

In [ ]:
for name in x1d_names:
    with fits.open(name, 'update') as hdulist:
        print(f'Updating error for {name}')
        for extnum in range(len(hdulist)):
            hdu = hdulist[extnum]
            if hdu.name == 'EXTRACT1D':
                print(f"Extension {extnum}")

                # report value before correction
                print(f"\tMean error before update: {np.nanmean(hdu.data['FLUX_ERROR'])}\n")

                # report input values
                print(f"\tMean Poisson error: {np.sqrt(np.nanmean(hdu.data['FLUX_VAR_POISSON']))}")
                print(f"\tMean read noise error: {np.sqrt(np.nanmean(hdu.data['FLUX_VAR_RNOISE']))}")
                print(f"\tMean flat error: {np.sqrt(np.nanmean(hdu.data['FLUX_VAR_FLAT']))}\n")
                
                # add components in quadrature, ignoring NaNs
                components = [hdu.data['FLUX_VAR_POISSON'], 
                              hdu.data['FLUX_VAR_RNOISE'], 
                              hdu.data['FLUX_VAR_FLAT']]
                new_error = np.sqrt(np.nansum(components, axis=0))
                
                # update FITS file
                hdu.data['FLUX_ERROR'] = new_error

                # report value after correction
                print(f"\tMean error after update: {np.nanmean(hdu.data['FLUX_ERROR'])}\n")


If the flat error has not been provided (i.e. the mean flat error is reported as zero or NaN, above), the calculated error may be underestimated for the spectrum.  If desired, an estimate of the systematic error due to flat correction can be added in place of the flat variance array.

For example, add a 1% overall error on the flux in place of the flat variance.

In [ ]:
for name in x1d_names:
    with fits.open(name, 'update') as hdulist:
        print(f'Updating error for {name} with additional estimated flat error')
        for extnum in range(len(hdulist)):
            hdu = hdulist[extnum]
            if hdu.name == 'EXTRACT1D':
                print(f"Extension {extnum}")

                # report value before correction
                print(f"\tMean error before update: {np.nanmean(hdu.data['FLUX_ERROR'])}\n")

                # report input values
                print(f"\tMean Poisson error: {np.sqrt(np.nanmean(hdu.data['FLUX_VAR_POISSON']))}")
                print(f"\tMean read noise error: {np.sqrt(np.nanmean(hdu.data['FLUX_VAR_RNOISE']))}")

                # estimate a 1% flat error
                flat_var = (0.01 * hdu.data['FLUX']) ** 2
                print(f"\tMean estimated flat error: {np.sqrt(np.nanmean(flat_var))}\n")
                
                # add components in quadrature, ignoring NaNs
                components = [hdu.data['FLUX_VAR_POISSON'], 
                              hdu.data['FLUX_VAR_RNOISE'], 
                              flat_var]
                new_error = np.sqrt(np.nansum(components, axis=0))
                
                # update FITS file
                hdu.data['FLUX_ERROR'] = new_error

                # report value after correction
                print(f"\tMean error after update: {np.nanmean(hdu.data['FLUX_ERROR'])}\n")

The 2D spectral files contain similar component variance information, stored in image extensions rather table columns. The error on the flux, in the ERR extension, generally was not impacted by the all-NaN propagation issue in the pipeline, but it may still be underestimated for some reference flat files.  An additional error may be added to these products in a similar manner.

Apply the same systematic 1% error to the s2d data products.

In [ ]:
for name in s2d_names:
    with fits.open(name, 'update') as hdulist:
        print(f'Updating error for {name}')
        # retrieve data from file, update flux_error column with nansum
        # print mean errors
